In [95]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, LeakyReLU
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop, Nadam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [128]:
df = pd.read_csv('train.csv')
tdf = pd.read_csv('test.csv')
print(df.shape, tdf.shape)
df

(8693, 14) (4277, 13)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [56]:
tdf

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [4]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [129]:
train_df = df.drop(columns=['Name', 'Cabin', 'PassengerId'])
test_tdf = tdf.drop(columns=['Name', 'Cabin', 'PassengerId'])
print(df.shape, tdf.shape)

(8693, 14) (4277, 13)


In [130]:
df_numeric = train_df.select_dtypes(include=['number'])
train_df.loc[:, df_numeric.columns] = df_numeric.fillna(df_numeric.mean())

tdf_numeric = test_tdf.select_dtypes(include=['number'])
test_tdf.loc[:, tdf_numeric.columns] = tdf_numeric.fillna(tdf_numeric.mean())

df_categorical = train_df.select_dtypes(include=['object', 'category'])
train_df.loc[:, df_categorical.columns] = df_categorical.apply(lambda x: x.fillna(x.mode()[0]))

tdf_categorical = test_tdf.select_dtypes(include=['object', 'category'])
test_tdf.loc[:, tdf_categorical.columns] = tdf_categorical.apply(lambda x: x.fillna(x.mode()[0]))
print(df.shape, tdf.shape)

(8693, 14) (4277, 13)


C:\Users\salir\AppData\Local\Temp\ipykernel_7044\2349466253.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df.loc[:, df_categorical.columns] = df_categorical.apply(lambda x: x.fillna(x.mode()[0]))
C:\Users\salir\AppData\Local\Temp\ipykernel_7044\2349466253.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_tdf.loc[:, tdf_categorical.columns] = tdf_categorical.apply(lambda x: x.fillna(x.mode()[0]))


In [22]:
len(df['Cabin'].unique())

6560

In [131]:
train_df = pd.get_dummies(train_df)
test_tdf = pd.get_dummies(test_tdf)
print(train_df.shape, test_tdf.shape)

(8693, 17) (4277, 16)


In [132]:
X = train_df.drop(columns='Transported')
y = train_df['Transported']
print(X.shape, y.shape)

(8693, 16) (8693,)


In [133]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X)
X_test = scaler.fit_transform(test_tdf)
print(X_train.shape, X_test.shape)

(8693, 16) (4277, 16)


In [44]:
model = Sequential()

model.add(Dense(10, activation='relu', input_shape=(16,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


c:\ML Algorithms\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           170 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │           550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771 (3.01 KB)

 Trainable params: 771 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
history = model.fit(X_train, y, epochs=150, validation_split=0.2, batch_size=32, verbose=1)

Epoch 1/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6256 - loss: 0.6493 - val_accuracy: 0.7890 - val_loss: 0.4750
Epoch 2/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7753 - loss: 0.4768 - val_accuracy: 0.7976 - val_loss: 0.4377
Epoch 3/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7791 - loss: 0.4639 - val_accuracy: 0.7907 - val_loss: 0.4312
Epoch 4/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7774 - loss: 0.4518 - val_accuracy: 0.7936 - val_loss: 0.4244
Epoch 5/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7919 - loss: 0.4346 - val_accuracy: 0.8016 - val_loss: 0.4250
Epoch 6/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7891 - loss: 0.4461 - val_accuracy: 0.7970 - val_loss: 0.4139
Epoch 7/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7803 - loss: 0.4507 - val_accuracy: 0.7947 - val_loss: 0.4167
Epoch 8/150
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7923 - loss: 0.4384 - val_accu

In [46]:
pred = model.predict(test_tdf)
pred

134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([[0.7751916 ],
       [0.        ],
       [0.97913265],
       ...,
       [0.9602292 ],
       [1.        ],
       [0.8809921 ]], dtype=float32)

In [59]:
pred = np.where(pred > 0.5, 1, 0)
pred = pred.flatten()
pred = pred.astype(bool)
pred

array([ True, False,  True, ...,  True,  True,  True])

In [60]:
submission = pd.DataFrame({
    'PassengerId' : tdf['PassengerId'],
    'Transported' : pred
})

submission.to_csv('submission.csv', index=False)

In [136]:
model2 = Sequential()

model2.add(Dense(100, activation='tanh', input_shape=(16,)))
#model2.add(BatchNormalization())
model2.add(Dense(200, activation='tanh'))
#model2.add(BatchNormalization())
#model2.add(Dense(, activation='relu'))
model2.add(Dense(150, activation='tanh'))
model2.add(Dense(1, activation='sigmoid'))



#optimizer = RMSprop(alpha=(0.1))

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_114 (Dense)               │ (None, 100)            │         1,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_115 (Dense)               │ (None, 200)            │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_116 (Dense)               │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_117 (Dense)               │ (None, 1)              │           151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,201 (203.91 KB)

 Trainable params: 52,201 (203.91 KB)

 Non-trainable params: 0 (0.00 B)

In [137]:
history2 = model2.fit(X_train, y, epochs=100, validation_split=0.2, batch_size=32, verbose=1)

Epoch 1/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7660 - loss: 0.4979 - val_accuracy: 0.7890 - val_loss: 0.4239
Epoch 2/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7844 - loss: 0.4602 - val_accuracy: 0.7959 - val_loss: 0.4291
Epoch 3/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7904 - loss: 0.4505 - val_accuracy: 0.7959 - val_loss: 0.4168
Epoch 4/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7844 - loss: 0.4547 - val_accuracy: 0.8022 - val_loss: 0.4188
Epoch 5/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7850 - loss: 0.4438 - val_accuracy: 0.7936 - val_loss: 0.4247
Epoch 6/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7923 - loss: 0.4444 - val_accuracy: 0.7999 - val_loss: 0.4229
Epoch 7/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7913 - loss: 0.4389 - val_accuracy: 0.8085 - val_loss: 0.4118
Epoch 8/100
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7916 - loss: 0.4395 - val_accu

In [138]:
pred2 = model2.predict(X_test)
pred2 = np.where(pred2 > 0.5, 1, 0)
pred2 = pred2.flatten()
pred2 = pred2.astype(bool)
pred2

134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


array([ True, False,  True, ...,  True,  True,  True])

In [139]:
submission = pd.DataFrame({
    'PassengerId' : tdf['PassengerId'],
    'Transported' : pred2
})

submission.to_csv('submission.csv', index=False)